# **8. 데이터프레임 결합하기**
1. 분석가로서 데이터 결합을 하는 경우
* 기존 데이터에 동일한 구조로 된 행 추가 = 수직결합(combining data vetically) = 이어 붙이기(concatenate)
* 다른 데이터 테이블에서 열을 조회하려고 병합 = 수평결합(combining data horizontally) = **병합(merging)**

2. **병합(merge)**의 종류
* '병합 기준 열 값이 얼마나 중복되는지'에 따라  
     1) 일대일(one-to-one) 병합 : 병합 기준 열 값이 각 데이터 테이블에 한 번씩 나타남  
     2) 일대다(one-to-many) 병합 : 한 쪽에는 중복되지 않은 병합 기준 열 값 존재, 다른 쪽에는 중복된 병합 기준 열 값 존재  
     3) 다대다(many-to-many) 병합 : 양쪽 모두에 병합 기준 열 값이 중복됨 

3. 데이터 결합에서 발생하는 이슈
- 한 쪽에서 병합 기준 누락되어, 추가되는 쪽 역시 값이 누락되는 문제 등.. 

## 8.1. 데이터프레임 수직 결합하기


In [2]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.width', 200)
pd.set_option('display.max_columns', 35)
pd.set_option('display.max_rows', 50)
pd.options.display.float_format = '{:,.0f}'.format

In [3]:
# 1. 지표온도 데이터 업로드
ltcameroon = pd.read_csv("data/ltcountry/ltcameroon.csv")
ltpoland = pd.read_csv("data/ltcountry/ltpoland.csv")

In [4]:
# 2. 데이터 이어붙이기(concatenate) : Cameroon - Poland data
ltcameroon.shape

(48, 11)

In [5]:
ltpoland.shape

(120, 11)

In [6]:
ltall = pd.concat([ltcameroon, ltpoland])
ltall.country.value_counts()

Poland      120
Cameroon     48
Name: country, dtype: int64

In [7]:
# 3. 데이터 이어붙이기 (concatenate) : 모든 국가별 파일의 데이터
directory = "data/ltcountry"
ltall = pd.DataFrame()
for filename in os.listdir(directory):
    if filename.endswith(".csv"): 
        fileloc = os.path.join(directory, filename)
        # 파일을 연다
        with open(fileloc) as f:
            ltnew = pd.read_csv(fileloc)
            print(filename + " has " + str(ltnew.shape[0]) + " rows.")
            ltall = pd.concat([ltall, ltnew])
        # 칼럼에 차이가 있는지 확인한다
        columndiff = ltall.columns.symmetric_difference(ltnew.columns)
        if (not columndiff.empty):
            print("", "Different column names for:", filename, columndiff, "", sep="\n")

ltbrazil.csv has 1104 rows.
ltcameroon.csv has 48 rows.
ltindia.csv has 1056 rows.
ltjapan.csv has 1800 rows.
ltmexico.csv has 852 rows.
ltoman.csv has 288 rows.

Different column names for:
ltoman.csv
Index(['latabs'], dtype='object')

ltpoland.csv has 120 rows.


In [8]:
# 4. 결합된 데이터 일부 표시
ltall[['country','station','month','temperature','latitude']].sample(5, random_state=1)

,country,station,month,temperature,latitude
717,Brazil,TAGUATINGA,8,28,-12
649,Japan,MATSUMOTO,5,17,36
172,Oman,BURAIMI_AUT,8,37,24
1029,India,JAIPUR_SANGANER,12,15,27
351,Mexico,SN_CRISTOBAL_LAS_CASASCHIS,5,19,17


In [9]:
# 5. 이어붙인 데이터 값 검사
ltall.country.value_counts().sort_index()
ltall.groupby(['country']).agg({'temperature':['min','mean',\
  'max','count'],'latabs':['min','mean','max','count']})

temperature                latabs               
                 min mean max count    min mean max count
country                                                  
Brazil            12   25  34   969      0   14  34  1104
Cameroon          22   27  36    34      4    8  10    48
India              2   26  37  1044      8   21  34  1056
Japan             -7   15  30  1797     24   36  45  1800
Mexico             7   23  34   806     15   22  32   852
Oman              12   28  38   205    nan  nan nan     0
Poland            -4   10  23   120     50   52  55   120

In [10]:
# 6. 누락값 수정
ltall['latabs'] = np.where(ltall.country=="Oman", ltall.latitude, ltall.latabs)
ltall.groupby(['country']).agg({'temperature':['min','mean','max','count'],'latabs':['min','mean','max','count']})

temperature                latabs               
                 min mean max count    min mean max count
country                                                  
Brazil            12   25  34   969      0   14  34  1104
Cameroon          22   27  36    34      4    8  10    48
India              2   26  37  1044      8   21  34  1056
Japan             -7   15  30  1797     24   36  45  1800
Mexico             7   23  34   806     15   22  32   852
Oman              12   28  38   205     17   22  26   288
Poland            -4   10  23   120     50   52  55   120